<a href="https://colab.research.google.com/github/ejrbdx/Advance-Research-Methods-2024/blob/main/GRU_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Advanced Research Methods/In class assignments/Class 3 25'

/content/drive/MyDrive/Advanced Research Methods/In class assignments/Class 3 25


In [3]:
import pandas as pd
import random, math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import fftpack
import torch.utils.data as utils
import os
from sklearn.metrics import mean_absolute_percentage_error
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [4]:
# !gdown 'https://drive.google.com/uc?id=1E_qqe7kfvfApM4hCOBMPoXhyEPyrUJkN'
# !gdown 'https://drive.google.com/uc?id=1j-3-lHegY--FDHKZvz86HBtilV2dsP-i'

In [5]:
# !unzip 'train.zip' -d '.'
# !unzip 'test.zip' -d '.'

In [6]:
# df_ = pd.read_pickle('tps_df.pkl')

In [7]:
# test = 0.2
# df_len =df_.shape[0]
# df_train, df_test = df_.iloc[0:df_len-int(test*df_len)],df_.iloc[df_len-int(test*df_len):]
# print (df_train.shape, df_test.shape)
# df_train.to_csv('train.csv',index = False)
# df_test.to_csv('test.csv',index = False)
# # df_train.shape,df_test.shape
# ## split it into training and test. 80%:20%

In [8]:
class TrafficForecast(Dataset):
    def __init__(self,csv_path, window, horizon):
        self.csv_path = csv_path
        self.window = window
        self.horizon = horizon
        self.df = None
        self.seg_ids = None
        self.inputs = []
        self.targets = []


        df = pd.read_csv(self.csv_path)
        df['time'] = pd.to_datetime(df['time'])
        df['unix_timestamp'] = df['time'].astype(int) / 10**9
        df['dow'] = df['time'].dt.weekday
        df['hour'] = df['time'].dt.hour
        df['min'] = df['time'].dt.minute
        self.df = df.sample(frac=1).reset_index(drop=True)
        # print (self.df.shape)



        self.seg_ids = self.df['segmentID'].unique()
        self.setup_forecast()

    def setup_forecast(self):
        for segid in self.seg_ids:
            df_seg_id = self.df[self.df['segmentID'] == segid]
            df_seg_id = df_seg_id.sort_values(by='time',ascending=True)
            df_seg_id = df_seg_id.fillna(method="ffill")
            TI_series = df_seg_id['TrafficIndex_GP'].values
            h_series = df_seg_id['hour'].values
            # print (len(TI_series))
            for t in range(0,len(TI_series)-(self.window+self.horizon)):
                x = TI_series[t:t+self.window]
                h = h_series[t:t+self.window]
                y = TI_series[t+self.window:(t+self.window+self.horizon)]

                xh = np.dstack([x,h])

                self.inputs.append(xh)
                self.targets.append(y)
        # print (self.inputs)

    def __len__(self):
        return len(self.inputs)


    def __getitem__(self,idx):
      X = torch.tensor(self.inputs[idx],dtype=torch.float32).reshape(self.window,2)
      y=torch.tensor(self.targets[idx],dtype=torch.float32)

      return {'inputs':X,'outputs':y}



In [9]:
tobj = TrafficForecast('train.csv',36,36)
tobj_test = TrafficForecast('test.csv',36,36)

In [10]:
bs = 1024
dataloader_train = DataLoader(tobj, batch_size=bs, shuffle=False, num_workers=4,drop_last=True)
dataloader_test = DataLoader(tobj_test, batch_size=bs, shuffle=False, num_workers=4,drop_last=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [11]:
for idx, data in enumerate(dataloader_test):
    print (data['inputs'].shape,data['outputs'].shape)
    if idx == 3:
        break

torch.Size([1024, 36, 2]) torch.Size([1024, 36])
torch.Size([1024, 36, 2]) torch.Size([1024, 36])
torch.Size([1024, 36, 2]) torch.Size([1024, 36])
torch.Size([1024, 36, 2]) torch.Size([1024, 36])


In [12]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
from models import MyLSTM, MyGRU

In [14]:

model = MyGRU(n_inputs=2, n_hidden=10, n_rnnlayers=2, n_outputs=36)
# model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
mean_loss = []
losses = []
test_losses = []
mean_test_loss = []
n_epochs = 3

for it in tqdm(range(n_epochs)):
  # zero the parameter gradients
  for i_batch, sample_batched in enumerate(dataloader_train):
    optimizer.zero_grad()
    outputs = model(sample_batched['inputs'])
    loss = criterion(outputs, sample_batched['outputs'])
    losses.append(loss.item())
    loss.backward()
    optimizer.step()

  for i_test,sample_test in enumerate(dataloader_test):

      outputs = model(sample_test['inputs'])
      test_loss = criterion(outputs,sample_test['outputs'])
      test_losses.append(test_loss.item())

  mean_loss.append(np.mean(losses))
  mean_test_loss.append(np.mean(test_losses))
  if (it+1) % 1 == 0:
    print(f'Epoch {it+1}/{n_epochs}, Training Loss: {np.mean(losses):.4f}, Testing Loss: {np.mean(test_losses):.4f}')

  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
 33%|███▎      | 1/3 [02:11<04:23, 131.92s/it]

Epoch 1/3, Training Loss: 0.0938, Testing Loss: 0.0046


 67%|██████▋   | 2/3 [04:25<02:12, 132.99s/it]

Epoch 2/3, Training Loss: 0.0536, Testing Loss: 0.0068


100%|██████████| 3/3 [06:41<00:00, 133.99s/it]

Epoch 3/3, Training Loss: 0.0401, Testing Loss: 0.0079


In [16]:
torch.save(model.state_dict(), 'grumodel.pt')

In [17]:
model_arch = model
model_arch.load_state_dict(torch.load('grumodel.pt'))
model_arch.eval()

MyGRU(
  (relu): ReLU()
  (rnn): GRU(2, 10, num_layers=2, batch_first=True)
  (fc): Linear(in_features=10, out_features=36, bias=True)
)

In [18]:
def evaluation(model_arch,dataloader_test, bs):
    rmse = []
    maes = []

    for i_test,sample_test in enumerate(dataloader_test):

        preds = model_arch(sample_test['inputs'])
        targets = sample_test['outputs']
        error = ((preds-targets)**2).sum(1).mean().detach().numpy()
        mae = (preds-targets).abs().sum(1).mean().detach().numpy()
        rmse.append(math.sqrt(error))
        maes.append(mae)
    return {'rmse':np.mean(rmse),'mae':np.mean(maes)}

In [19]:
results = evaluation(model_arch,dataloader_test,bs)

In [21]:
print(results)

{'rmse': 0.5975147327956473, 'mae': 3.411336}
